### Passo 1 - Importar Arquivos e Bibliotecas

In [1]:
def money_print(text):
    text = '{:,.2f}'.format(text)
    text = text.replace(',','_').replace('.',',').replace('_','.')
    return text

In [35]:
import pandas as pd
import os
import yagmail

In [3]:
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', delimiter = ';', encoding='latin1')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

In [4]:
display(emails[:3])
display(lojas[:3])
display(vendas[:3])

,Loja,Gerente,E-mail,Unnamed: 3,Unnamed: 4
0,Iguatemi Esplanada,Helena,pythonimpressionador+helena@gmail.com,NaN,NaN
1,Shopping Midway Mall,Alice,pythonimpressionador+alice@gmail.com,NaN,NaN
2,Norte Shopping,Laura,pythonimpressionador+laura@gmail.com,NaN,NaN


,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368


### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [5]:
vendas = vendas.merge(lojas, left_on='ID Loja', right_on='ID Loja' )
display(vendas[:3])

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada


In [6]:
dic_lojas = {}
for loja in lojas['Loja']:
    dic_lojas[loja] = vendas.loc[vendas['Loja'] == loja, :]

In [7]:
dia_indicador = vendas['Data'].max()
dia_indicador.day

26

### Passo 3 - Salvar a planilha na pasta de backup

In [8]:
caminho_backup = r'Backup Arquivos Lojas'

os.path.exists(caminho_backup)

for loja in dic_lojas:
    pasta_loja= caminho_backup + '\\' + loja
    if not(os.path.exists(pasta_loja)):
        os.mkdir(pasta_loja)
    nome_arquivo = f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_{loja}.xlsx'
    caminho_arquivo = pasta_loja + '\\'+nome_arquivo
    if not(os.path.exists(caminho_arquivo)):
        dic_lojas[loja].to_excel(caminho_arquivo)



### Passo 4 - Calcular o indicador para 1 loja

Faturamento anual = R$ 1.711.968,00
Faturamento dia = R$ 1.259,00
Variedade de 120 produtos ao ano
Variedade de 2 produtos no dia
Ticket médio anual = R$784,59
Ticket médio dia = R$1.259,00


In [11]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1_650_000
meta_qtd_produtos_ano = 120
meta_qtd_produtos_dia = 500
meta_ticket_medio_dia =1000
meta_ticket_medio_ano = 100

### Passo 5 - Enviar por e-mail para o gerente

### Passo 6 - Automatizar todas as lojas

In [28]:
usuario = yagmail.SMTP(user = 'gabriel.tavarez@usp.br', password = '@123quatro')

for loja in dic_lojas:
    ########CALCULO INDICADORES#############3
    
    loja_df = dic_lojas[loja]
    loja_df_dia = loja_df.loc[dic_lojas[loja]['Data'] == dia_indicador]

    #faturamento
    faturamento_ano = loja_df['Valor Final'].sum()
    faturamento_dia = loja_df_dia['Valor Final'].sum()

    #diversidade de produtos
    produtos_ano = loja_df['Produto'].unique()
    produtos_dia = loja_df_dia['Produto'].unique()


    #ticket medio
    agrupamento_vendas_ano = loja_df.groupby('Código Venda').sum()
    ticket_medio_ano = agrupamento_vendas_ano['Valor Final'].mean()

    agrupamento_vendas_dia = loja_df_dia.groupby('Código Venda').sum()
    ticket_medio_dia = agrupamento_vendas_dia['Valor Final'].mean()


    ##########ENVIAR EMAIL############3
    destinatario = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
    assunto = f"OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}"

    nome =  emails.loc[emails['Loja']==loja, 'Gerente'].values[0]

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'
    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'
    if qtde_produtos_dia >= meta_qtdeprodutos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'
    if qtde_produtos_ano >= meta_qtdeprodutos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'
    if ticket_medio_dia >= meta_ticketmedio_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'
    if ticket_medio_ano >= meta_ticketmedio_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'
    
    corpo_email = f'''
    <p>Bom dia, {nome}</p>

    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenário Dia</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
      </tr>
    </table>
    <br>
    <table>
      <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenário Ano</th>
      </tr>
      <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
      </tr>
      <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
      </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou à disposição.</p>
    <p>Att., Lira</p>'''


    anexo = os.getcwd() + '\\' + caminho_backup + '\\' +  f'{dia_indicador.day}_{dia_indicador.month}_{dia_indicador.year}_{loja}.xlsx'

    usuario.send(to=destinatario, subject=assunto, contents = corpo_email, \
                attachments = anexo)


### Passo 7 - Criar ranking para diretoria

'C:\\Users\\gabri\\OneDrive\\Documents\\Cursos\\Python Completo\\Projeto AutomacaoIndicadores\\Backup Arquivos Lojas\\26_12_2019_Norte Shopping.xlsx'

### Passo 8 - Enviar e-mail para diretoria

Iguatemi Esplanada
Shopping Midway Mall
Norte Shopping
Shopping Iguatemi Fortaleza
Shopping União de Osasco
Shopping Center Interlagos
Rio Mar Recife
Salvador Shopping
Rio Mar Shopping Fortaleza
Shopping Center Leste Aricanduva
Ribeirão Shopping
Shopping Morumbi
Parque Dom Pedro Shopping
Bourbon Shopping SP
Palladium Shopping Curitiba
Passei das Águas Shopping
Center Shopping Uberlândia
Shopping Recife
Shopping Vila Velha
Shopping SP Market
Shopping Eldorado
Shopping Ibirapuera
Novo Shopping Ribeirão Preto
Iguatemi Campinas
Shopping Barra
